In [8]:
import pandas as pd
df_games = pd.read_csv('steam_games_clean.csv')
df_items = pd.read_csv('users_items_clean.csv')
df_reviews = pd.read_csv('users_reviews_clean.csv')

## Endpoint 1

In [2]:
df_games["developer"].value_counts()

developer
Ubisoft - San Francisco       1258
SmiteWorks USA, LLC            812
Dovetail Games                 253
KOEI TECMO GAMES CO., LTD.     214
Unknown                        155
                              ... 
Project Purity Team              1
Triple rush games                1
Storybird,Julien Rocca           1
Light                            1
Bidoniera Games                  1
Name: count, Length: 10590, dtype: int64

In [3]:
df_games["año"] = df_games["release_year"]  #Creo una columna de año.

contenido_free = df_games[df_games["price"] == 0.0] #Selecciono un subset donde los precios sean 0.0.

total_juegos_por_desarrollador = df_games.groupby(["developer", "año"]).size().reset_index(name="total_juegos") #Agrupo a los juegos por año y desarrollador.
num_juegos_gratuitos = contenido_free.groupby(["developer", "año"]).size().reset_index(name='num_juegos_gratuitos')

juegos_por_desarrollador = total_juegos_por_desarrollador.merge(num_juegos_gratuitos, on=["developer", "año"], how="left") #Uno en un dataframe los juegos free y los demás.
juegos_por_desarrollador["porcentaje_juegos_gratuitos"] = (juegos_por_desarrollador["num_juegos_gratuitos"] / juegos_por_desarrollador["total_juegos"]) * 100 #Calculo el promedio de juegos gratuitos

juegos_por_desarrollador = juegos_por_desarrollador.fillna(0)
juegos_por_desarrollador = juegos_por_desarrollador.drop("num_juegos_gratuitos", axis=1)
juegos_por_desarrollador["porcentaje_juegos_gratuitos"] = juegos_por_desarrollador["porcentaje_juegos_gratuitos"].apply(lambda x: f'{int(x)}%')

In [4]:
#Chequeo que haya quedado bien.
juegos_por_desarrollador[juegos_por_desarrollador["developer"]=="Dovetail Games"]

,developer,año,total_juegos,porcentaje_juegos_gratuitos
3597,Dovetail Games,2009,22,0%
3598,Dovetail Games,2010,4,0%
3599,Dovetail Games,2011,4,0%
3600,Dovetail Games,2012,52,0%
3601,Dovetail Games,2013,39,0%
3602,Dovetail Games,2014,37,0%
3603,Dovetail Games,2015,50,0%
3604,Dovetail Games,2016,21,0%
3605,Dovetail Games,2017,24,4%


In [5]:
juegos_por_desarrollador["porcentaje_juegos_gratuitos"].value_counts()

porcentaje_juegos_gratuitos
0%      13113
100%      786
50%       237
33%        72
25%        71
20%        26
14%        19
66%        16
16%        16
12%        12
9%          7
28%         6
7%          6
11%         6
4%          5
5%          4
6%          4
2%          4
22%         3
75%         3
8%          2
42%         2
41%         2
37%         2
40%         2
3%          1
35%         1
55%         1
17%         1
13%         1
85%         1
18%         1
87%         1
26%         1
1%          1
10%         1
15%         1
46%         1
Name: count, dtype: int64

In [6]:
#Almaceno el df en un csv para ser consumido por la API.
juegos_por_desarrollador.to_csv("df_endpoint1.csv", index=False)

## Endpoint 2

In [2]:
columnas = ['id', 'price']
df_precios = df_games[columnas].copy()

In [3]:
df_precios.head()

,id,price
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,772540.0,3.99


In [4]:
df_precios.rename(columns={'id': 'item_id'}, inplace=True)

In [5]:
df_precios

,item_id,price
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,772540.0,3.99
...,...,...
27557,745400.0,1.99
27558,773640.0,1.99
27559,733530.0,4.99
27560,610660.0,1.99


In [11]:
df_items["items"][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

In [18]:
df_items = df_items.head(5000)

In [19]:
#Función para calcular el gasto de cada usuario
def calcular_gasto(user):
    gasto_total = 0
    for game in user:
        item_id = int(game['item_id'])
        precio_fila = df_precios.loc[df_precios['item_id'] == item_id]  
        if not precio_fila.empty:
            precio = precio_fila["price"].values[0]
            gasto_total += precio
    return gasto_total

df_items['gasto_total'] = df_items['items'].apply(calcular_gasto)

In [21]:
df_endpoint2 = df_items[['user_id', 'items_count', 'gasto_total']]

In [22]:
df_endpoint2

,user_id,items_count,gasto_total
0,76561197970982479,277,3399.33
1,js41637,888,8367.41
2,evcentric,137,1579.91
3,Riot-Punch,328,3284.47
4,doctr,541,6655.59
...,...,...,...
4995,76561198051265484,4,99.97
4996,76561198097453305,25,169.88
4997,76561198096899089,31,252.84
4998,76561198022316167,123,1382.73


In [23]:
#Guardo el df en un csv para ser consumido por la API.
df_endpoint2.to_csv("df_endpoint2.csv", index=False)

## Endpoint 3

In [26]:
small_games = df_games.head(5000)
small_items = df_items.head(1000)

In [28]:
def UserForGenre(genre):
    user_name = ''
    max_hours_count = 0
    for user in small_items.user_id:
        hours_count = 0
        for items in small_items.loc[small_items['user_id'] == user, 'items'].values[0]:
            game_id = items.get('item_id')
            if int(game_id) in df_games['id'].values:
                genres = df_games.loc[df_games['id'] == int(game_id)].values[0]
                if str(genre) in str(genres):
                    hours_count += int(items.get('playtime_forever'))

        if hours_count > max_hours_count:
            max_hours_count = hours_count
            user_name = user
    return "Usuario con más horas jugadas para Género {}: {} con {} horas.".format(genre, user_name, max_hours_count)

genres_list = ['Action','Simulator','RPG','Indie','Adventure','Casual','Strategy','Racing']

lista = []

# Itero la funcion en cada genero
for genre in genres_list:
    result = UserForGenre(genre)
    lista.append({'genre': genre, 'result': result})

df_endpoint3 = pd.DataFrame(lista)
df_endpoint3.to_csv('df_endpoint3.csv', index=False)

In [29]:
df_endpoint3

,genre,result
0,Action,Usuario con más horas jugadas para Género Acti...
1,Simulator,Usuario con más horas jugadas para Género Simu...
2,RPG,Usuario con más horas jugadas para Género RPG:...
3,Indie,Usuario con más horas jugadas para Género Indi...
4,Adventure,Usuario con más horas jugadas para Género Adve...
5,Casual,Usuario con más horas jugadas para Género Casu...
6,Strategy,Usuario con más horas jugadas para Género Stra...
7,Racing,Usuario con más horas jugadas para Género Raci...


## Endpoint 4

In [30]:
small_reviews_df = df_reviews.head(20000)

In [31]:
small_games.head()

,genres,title,price,id,developer,release_year,gasto_total
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018,3399.33
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,0.00,643980.0,Secret Level SRL,2018,8367.41
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,2017,1579.91
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,0.99,767400.0,彼岸领域,2017,3284.47
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,2018,6655.59


In [38]:
from collections import Counter
def best_developer_year(año):
    developers = []
    for review in small_reviews_df.reviews:
        for review_i in review:
            if str(año) in review_i.get('posted'):
                item_id = review_i.get('item_id')
                for id in small_games.id:
                    if int(id) == int(item_id):
                        if review_i.get('recommend') == True:
                            developer = small_games.loc[small_games['id'] == id, 'developer'].values[0]
                            developers.append(developer)                  

    developers_counter = Counter(developers)
    top_3_developers = [developer for developer, _ in developers_counter.most_common(3)]
    return [{"Puesto {}: {}".format(i+1, developer)} for i, developer in enumerate(top_3_developers)]

In [42]:
años = range(2009, 2018)
developers = []
for año in años:
    developer = best_developer_year(año)
    developers.append({'año': año, 'developers': developer})

df_endpoint4 = pd.DataFrame(developers)
df_endpoint4.to_csv('df_endpoint4.csv', index=False)

In [43]:
df_endpoint4

,año,developers
0,2009,[]
1,2010,"[{Puesto 1: Valve}, {Puesto 2: Facepunch Studi..."
2,2011,"[{Puesto 1: Valve}, {Puesto 2: Facepunch Studi..."
3,2012,"[{Puesto 1: Valve}, {Puesto 2: Facepunch Studi..."
4,2013,"[{Puesto 1: Facepunch Studios}, {Puesto 2: Val..."
5,2014,"[{Puesto 1: Valve}, {Puesto 2: Facepunch Studi..."
6,2015,"[{Puesto 1: Valve}, {Puesto 2: Facepunch Studi..."
7,2016,[]
8,2017,[]


## Endpoint 5

In [44]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [51]:
df_reviews["reviews"][4]

[{'funny': '3 people found this review funny',
  'posted': 'Posted April 15, 2014.',
  'last_edited': '',
  'item_id': '211420',
  'helpful': '35 of 43 people (81%) found this review helpful',
  'recommend': True,
  'review': 'Git gud'},
 {'funny': '1 person found this review funny',
  'posted': 'Posted December 23, 2013.',
  'last_edited': '',
  'item_id': '211820',
  'helpful': '12 of 16 people (75%) found this review helpful',
  'recommend': True,
  'review': "It's like Terraria, you play for 9 hours straight, get endgame armour then stop playing until the next update."},
 {'funny': '2 people found this review funny',
  'posted': 'Posted March 14, 2014.',
  'last_edited': '',
  'item_id': '730',
  'helpful': '5 of 5 people (100%) found this review helpful',
  'recommend': True,
  'review': 'Hold shift to win, Hold CTRL to lose.'},
 {'funny': '',
  'posted': 'Posted July 11, 2013.',
  'last_edited': '',
  'item_id': '204300',
  'helpful': 'No ratings yet',
  'recommend': True,
  'rev

In [52]:
df_reviews = pd.DataFrame(df_reviews["reviews"])

In [53]:
from textblob import TextBlob
def analyze_sentiment(reviews_list):
    sentiments = []
    for review_dict in reviews_list:
        analysis = TextBlob(review_dict['review'])
        polarity = analysis.sentiment.polarity
        if polarity < 0:
            sentiments.append(0) 
        elif polarity == 0:
            sentiments.append(1)  
        else:
            sentiments.append(2) 
    return sentiments

In [54]:
df_reviews['sentiment_analysis'] = df_reviews['reviews'].apply(analyze_sentiment)

In [55]:
df_reviews.iloc[1]

reviews               [{'funny': '', 'posted': 'Posted June 24, 2014...
sentiment_analysis                                            [2, 0, 0]
Name: 1, dtype: object

In [56]:
df_reviews["reviews"][37]

[{'funny': '',
  'posted': 'Posted January 6, 2015.',
  'last_edited': '',
  'item_id': '213670',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'My character is a Jew named ♥♥♥♥♥♥♥♥♥ that is wearing a KKK costume.10/10, IGN'}]

In [57]:
import re
def sentiment_analysis(year):
    df_filtered = df_reviews[df_reviews['reviews'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

    pattern = r'Posted (\w+ \d{1,2}, \d{4}).'
    def extract_year(review):
        match = re.search(pattern, review[0]['posted'])
        if match:
            return int(match.group(1).split()[-1])
        else:
            return None

    df_filtered['posted_year'] = df_filtered['reviews'].apply(extract_year)

    filtered_reviews = df_filtered[df_filtered['posted_year'] == year]

    sentiment_counts = filtered_reviews['sentiment_analysis'].explode().value_counts().to_dict()

    sentiment_labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    sentiment_counts = {sentiment_labels[key]: value for key, value in sentiment_counts.items()}

    return sentiment_counts

In [58]:
years = list(range(2010, 2016))
results_dict = {'Year': [], 'Sentiment Analysis': []}

for year in years:
    sentiment_result = sentiment_analysis(year)
    results_dict['Year'].append(year)
    results_dict['Sentiment Analysis'].append(sentiment_result)

results_df = pd.DataFrame(results_dict)

C:\Users\lucas\AppData\Local\Temp\ipykernel_4456\1213327516.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['posted_year'] = df_filtered['reviews'].apply(extract_year)
C:\Users\lucas\AppData\Local\Temp\ipykernel_4456\1213327516.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['posted_year'] = df_filtered['reviews'].apply(extract_year)
C:\Users\lucas\AppData\Local\Temp\ipykernel_4456\1213327516.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [60]:
results_df

,Year,Sentiment Analysis
0,2009,{}
1,2010,"{'Positive': 15, 'Negative': 5, 'Neutral': 1}"
2,2011,"{'Positive': 164, 'Negative': 50, 'Neutral': 35}"
3,2012,"{'Positive': 423, 'Negative': 119, 'Neutral': 80}"
4,2013,"{'Positive': 3198, 'Neutral': 1149, 'Negative'..."
5,2014,"{'Positive': 12493, 'Neutral': 4701, 'Negative..."
6,2015,"{'Positive': 10756, 'Negative': 4426, 'Neutral..."


In [61]:
results_df.to_csv("df_endpoint5.csv",index=False)